In [2]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
import scipy.stats
import os
import scipy.io as sio
#import dnatools
#import re
import regex as re
from collections import Counter, defaultdict
#from pylab import *
#import matplotlib.pyplot as plt
import sys 
#%matplotlib inline

def distance(astring, bstring) :
    distance = 0
    
    limit = len(astring)
    diff = len(bstring) - len(astring)
    if len(bstring) < len(astring) :
        limit = len(bstring)
        diff = len(astring) - len(bstring)
    
    for i in range(limit) :
        if astring[i] != bstring[i] :
            distance += 1
    return distance + diff

C:\Users\Johannes\Anaconda3\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
r1_rna = 'Undetermined_S0_R1_001.fastq'
#r2_rna = 'Undetermined_S0_R2_001.fastq'
r_indx = 'Undetermined_S0_I1_001.fastq'

proximal_regex = re.compile(r"(AAAAAAAAAAAAAAAAAAAA){s<=3}")

wildtype_downstream_regex = re.compile(r"(GATGTCTCGTGATCTGGTGT){s<=2}")

upstream_regex = re.compile(r"(CAATTCTGCT){s<=2}[ACGTN]{40}(CTAAAATATA){s<=2}")
downstream_regex = re.compile(r"(AGTATGAAAC){s<=2}[ACGTN]{20}(ACCCTTATCC){s<=2}")
seq_regex = re.compile(r"(CAATTCTGCT){s<=2}[ACGTN]{40}(CTAAAATATA){s<=2}.*(AGTATGAAAC){s<=2}[ACGTN]{20}(ACCCTTATCC){s<=2}")

In [6]:
f1 = open(r1_rna,'r')
i1 = open(r_indx, 'r')

#f2 = open(r2_rna,'r')

head, seq, pr, q, head2, seq2, pr2, q2, headi, seqi, pri, qi = ({} for i in range(12))
count = 0

read_upstream = []
read_downstream = []
read_seq = []
read_umi = []
read_polya = []
read_prox = []

total_proximal_rna_count = 0
num_upstream_region_extractions = 0
num_downstream_region_extractions = 0

print('Processing RNA reads.')

out = open('apa_nextseq_v2_rna_20160619.csv','w')
out.write('upstream_seq,downstream_seq,seq,umi,polya,is_proximal\n')

while True:
    head = f1.readline()[:-1]
    seq = f1.readline()[:-1]
    pr = f1.readline()[:-1]
    q = f1.readline()[:-1]
        
    headi = i1.readline()[:-1]
    seqi = i1.readline()[:-1]
    pri = i1.readline()[:-1]
    qi = i1.readline()[:-1]
    
    if len(q) == 0:
        break # End of File
    
    upstream_flank = re.search(upstream_regex, seq)
    downstream_flank = re.search(downstream_regex, seq[70:220])
    both_flank = re.search(seq_regex, seq)

    if upstream_flank != None:
        num_upstream_region_extractions += 1
        upstream_flank_seq = upstream_flank.group()
        
        proximal_test_outcome = re.search(proximal_regex, seq)
        umi = seqi
        
        polya_pos = -1
        downstream_flank_seq = ''
        is_prox = 0
        if downstream_flank != None :
            num_downstream_region_extractions += 1
            downstream_flank_seq = downstream_flank.group()
        elif proximal_test_outcome != None :
            total_proximal_rna_count += 1
            polya_pos = proximal_test_outcome.start()
            is_prox = 1
        
        both_flank_seq = ''
        if both_flank != None :
            both_flank_seq = both_flank.group()
        
        #read_upstream.append(upstream_flank_seq)
        out.write(upstream_flank_seq)
        #read_downstream.append(downstream_flank_seq)
        out.write(',' + downstream_flank_seq)
        #read_seq.append(both_flank_seq)
        out.write(',' + both_flank_seq)
        #read_umi.append(umi)
        out.write(',' + umi)
        #read_polya.append(polya_pos)
        out.write(',' + str(polya_pos))
        #read_prox.append(is_prox)
        out.write(',' + str(is_prox))
        out.write('\n')
        
    if count % 1000000 == 0:
        print('Count: ' + str(count))
        print('Number of upstream regions extracted: ' + str(num_upstream_region_extractions))
        print('Number of downstream regions extracted: ' + str(num_downstream_region_extractions))
        print(str(total_proximal_rna_count) + ' proximal RNA reads')
    count += 1
    
print('COMPLETE')
print('Number of upstream regions extracted: ' + str(num_upstream_region_extractions))
print('Number of downstream regions extracted: ' + str(num_downstream_region_extractions))
print(str(total_proximal_rna_count) + ' proximal RNA reads')

out.close()

f1.close()
#f2.close()
i1.close()

'''df = pd.DataFrame({'upstream_seq':   read_upstream, 
                   'downstream_seq':   read_downstream, 
                   'seq':         read_seq
                   'umi':         read_umi,
                   'polya':       read_polya,
                   'is_proximal': read_prox})

new_columns = ['upstream_seq', 'downstream_seq', 'seq', 'umi', 'polya', 'is_proximal']
df.to_csv('apa_nextseq_v2_rna_20160619.csv', sep=',', header=True, columns=new_columns, index=False)'''

Processing RNA reads.
Count: 0
Number of upstream regions extracted: 1
Number of downstream regions extracted: 1
0 proximal RNA reads
Count: 1000000
Number of upstream regions extracted: 1000001
Number of downstream regions extracted: 559603
355985 proximal RNA reads
Count: 2000000
Number of upstream regions extracted: 2000001
Number of downstream regions extracted: 1128738
701108 proximal RNA reads
Count: 3000000
Number of upstream regions extracted: 3000001
Number of downstream regions extracted: 1690313
1041221 proximal RNA reads
Count: 4000000
Number of upstream regions extracted: 4000001
Number of downstream regions extracted: 2245020
1381469 proximal RNA reads
Count: 5000000
Number of upstream regions extracted: 5000001
Number of downstream regions extracted: 2805268
1726335 proximal RNA reads
Count: 6000000
Number of upstream regions extracted: 6000001
Number of downstream regions extracted: 3377419
2072526 proximal RNA reads
Count: 7000000
Number of upstream regions extracted: 

"df = pd.DataFrame({'upstream_seq':   read_upstream, \n                   'downstream_seq':   read_downstream, \n                   'seq':         read_seq\n                   'umi':         read_umi,\n                   'polya':       read_polya,\n                   'is_proximal': read_prox})\n\nnew_columns = ['upstream_seq', 'downstream_seq', 'seq', 'umi', 'polya', 'is_proximal']\ndf.to_csv('apa_nextseq_v2_rna_20160619.csv', sep=',', header=True, columns=new_columns, index=False)"

In [1]:
'1234'[:-1]

'123'

In [3]:
proximal_regex = re.compile(r"(AAAAAAAAAAAAAAAAAAAA){s<=3}")

test_re = re.search(proximal_regex, 'TTTAAGTTTTTTTGATAGTAAGGCCCATTACCTGAGGCCGCAATTCTGCTTGTTAAGAACAATCCCAGTTCTGGTAACTGACCTTCAAAGCTAAAATATAAAACTATTTGGGAAGTATGAAAAAAAAAAAAAAAAAAAAACCGGTTTCCGGATGGGGAGGGCGCCCGGGGGGGGGGCGGGCCCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG')

print(test_re.start())

115
